In [1]:
import librosa
import librosa.display
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
import seaborn as sns
import glob 
import os
from tqdm import tqdm
import pickle
import IPython.display as ipd

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [5]:
def noise(data):
    noise_amp = 0.005*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

def shift(data):
    s_range = int(np.random.uniform(low=-5, high = 5)*500)
    return np.roll(data, s_range)

def stretch(data, rate=0.8):
    return librosa.effects.time_stretch(data, rate)

def pitch(data, sampling_rate, pitch_factor=0.7):
    return librosa.effects.pitch_shift(data, sampling_rate, pitch_factor)

def dyn_change(data):
    dyn_change = np.random.uniform(low=1.5,high=3)
    return (data * dyn_change)

def speedNpitch(data):
    length_change = np.random.uniform(low=0.8,high=1)
    speed_fac = 1.0  / length_change
    tmp = np.interp(np.arange(0,len(data),speed_fac),np.arange(0,len(data)),data)
    minlen = min(data.shape[0], tmp.shape[0])
    data *= 0
    data[0:minlen] = tmp[0:minlen]
    return data

In [6]:
ref = pd.read_csv('Data_path.csv')
ref.head()

,labels,source,path
0,male_angry,SAVEE,SAVEE/ALL/DC_a01.wav
1,male_angry,SAVEE,SAVEE/ALL/DC_a02.wav
2,male_angry,SAVEE,SAVEE/ALL/DC_a03.wav
3,male_angry,SAVEE,SAVEE/ALL/DC_a04.wav
4,male_angry,SAVEE,SAVEE/ALL/DC_a05.wav


In [7]:
df = pd.DataFrame(columns=['feature'])
df_noise = pd.DataFrame(columns=['feature'])
df_speedpitch = pd.DataFrame(columns=['feature'])
cnt = 0

In [8]:
for i in tqdm(ref.path):
    X, sample_rate = librosa.load(i, res_type='kaiser_fast',duration=2.5,sr=44100,offset=0.5)
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=np.array(sample_rate), n_mfcc=13),axis=0)
    df.loc[cnt] = [mfccs]   
    aug = noise(X)
    aug = np.mean(librosa.feature.mfcc(y=aug, sr=np.array(sample_rate), n_mfcc=13), axis=0)
    df_noise.loc[cnt] = [aug]
    aug = speedNpitch(X)
    aug = np.mean(librosa.feature.mfcc(y=aug, sr=np.array(sample_rate), n_mfcc=13), axis=0)
    df_speedpitch.loc[cnt] = [aug]   
    cnt += 1

df.head()

100%|██████████| 12162/12162 [05:43<00:00, 35.36it/s]


,feature
0,"[-4.6414213, -3.8608985, -6.21919, -5.9265423,..."
1,"[-8.690717, -12.522839, -22.928043, -23.243807..."
2,"[-8.81486, -12.819055, -24.178183, -23.84745, ..."
3,"[-2.2684255, -4.3170767, -12.285238, -13.08302..."
4,"[-13.485307, -16.26042, -25.884357, -27.827045..."


In [9]:
df = pd.concat([ref,pd.DataFrame(df['feature'].values.tolist())],axis=1)
df_noise = pd.concat([ref,pd.DataFrame(df_noise['feature'].values.tolist())],axis=1)
df_speedpitch = pd.concat([ref,pd.DataFrame(df_speedpitch['feature'].values.tolist())],axis=1)
df.shape, df_noise.shape, df_speedpitch.shape

((12162, 219), (12162, 219), (12162, 219))

In [10]:
df = pd.concat([df,df_noise,df_speedpitch],axis=0,sort=False)
df=df.fillna(0)
del df_noise, df_speedpitch

df.head()

,labels,source,path,0,1,2,3,4,5,6,...,206,207,208,209,210,211,212,213,214,215
0,male_angry,SAVEE,SAVEE/ALL/DC_a01.wav,-4.641421,-3.860898,-6.219190,-5.926542,-5.850419,-4.808960,-2.513003,...,-4.088851,-5.023864,-5.254714,-5.234093,-5.310306,-5.621665,-6.072197,-6.611348,-3.999875,1.390507
1,male_angry,SAVEE,SAVEE/ALL/DC_a02.wav,-8.690717,-12.522839,-22.928043,-23.243807,-22.926605,-23.432243,-14.830002,...,-22.627256,-22.633406,-22.511599,-24.300154,-24.496809,-22.895985,-23.511505,-24.342152,-16.465857,-8.936036
2,male_angry,SAVEE,SAVEE/ALL/DC_a03.wav,-8.814860,-12.819055,-24.178183,-23.847450,-15.182783,-10.732485,-8.681472,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,male_angry,SAVEE,SAVEE/ALL/DC_a04.wav,-2.268425,-4.317077,-12.285238,-13.083024,-12.041327,-11.819768,-9.414147,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,male_angry,SAVEE,SAVEE/ALL/DC_a05.wav,-13.485307,-16.260420,-25.884357,-27.827045,-27.593534,-26.666512,-18.659025,...,-25.291666,-25.854906,-26.821354,-25.436455,-24.179941,-23.281618,-24.167494,-25.228062,-20.599659,-15.929615


In [11]:
df.isna().sum()

labels    0
source    0
path      0
0         0
1         0
         ..
211       0
212       0
213       0
214       0
215       0
Length: 219, dtype: int64

In [12]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(['path','labels', 'source'],axis=1), df.labels, test_size=0.25, shuffle=True, random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((27364, 216), (9122, 216), (27364,), (9122,))

In [13]:
mean = np.mean(x_train, axis=0)
std = np.std(x_train, axis=0)

x_train = (x_train - mean)/std
x_test = (x_test - mean)/std

x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [14]:
lb = LabelEncoder()
y_train = tf.keras.utils.to_categorical(lb.fit_transform(y_train))
y_test = tf.keras.utils.to_categorical(lb.fit_transform(y_test))
y_train.shape, y_test.shape

((27364, 14), (9122, 14))

In [15]:
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)
x_train.shape, x_test.shape

((27364, 216, 1), (9122, 216, 1))

In [23]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(256, 8, padding='same', activation='relu', input_shape=(x_train.shape[1],1)),
    tf.keras.layers.Conv1D(256, 8, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.MaxPooling1D(pool_size=(8)),
    tf.keras.layers.Conv1D(128, 8, padding='same', activation='relu'),
    tf.keras.layers.Conv1D(128, 8, padding='same', activation='relu'),
    tf.keras.layers.Conv1D(128, 8, padding='same', activation='relu'),
    tf.keras.layers.Conv1D(128, 8, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.MaxPooling1D(pool_size=(8)),
    tf.keras.layers.Conv1D(64, 8, padding='same', activation='relu'),
    tf.keras.layers.Conv1D(64, 8, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(14, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001, decay=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_8 (Conv1D)           (None, 216, 256)          2304      
                                                                 
 conv1d_9 (Conv1D)           (None, 216, 256)          524544    
                                                                 
 batch_normalization_8 (Batc  (None, 216, 256)         1024      
 hNormalization)                                                 
                                                                 
 dropout_10 (Dropout)        (None, 216, 256)          0         
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 27, 256)          0         
 1D)                                                             
                                                                 
 conv1d_10 (Conv1D)          (None, 27, 128)          

c:\Users\Fadil\anaconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [24]:
model.fit(x_train, y_train, batch_size=32, epochs=150, validation_data=(x_test, y_test), verbose=1)

Epoch 1/150
856/856 [==============================] - 14s 15ms/step - loss: 2.0717 - accuracy: 0.2770 - val_loss: 2.4192 - val_accuracy: 0.2258
Epoch 2/150
856/856 [==============================] - 13s 16ms/step - loss: 1.7780 - accuracy: 0.3693 - val_loss: 2.3334 - val_accuracy: 0.2841
Epoch 3/150
856/856 [==============================] - 13s 15ms/step - loss: 1.6702 - accuracy: 0.4122 - val_loss: 2.2549 - val_accuracy: 0.3267
Epoch 4/150
856/856 [==============================] - 12s 14ms/step - loss: 1.5939 - accuracy: 0.4345 - val_loss: 1.9006 - val_accuracy: 0.3671
Epoch 5/150
856/856 [==============================] - 12s 15ms/step - loss: 1.5417 - accuracy: 0.4534 - val_loss: 2.0360 - val_accuracy: 0.3476
Epoch 6/150
856/856 [==============================] - 13s 15ms/step - loss: 1.4915 - accuracy: 0.4721 - val_loss: 1.7480 - val_accuracy: 0.3965
Epoch 7/150
856/856 [==============================] - 13s 15ms/step - loss: 1.4504 - accuracy: 0.4874 - val_loss: 1.9699 - val_ac

In [25]:
model.save('model2.h5')